In [1]:
import pyzed.sl as sl

# importing required libraries
from PyQt5.QtWidgets import *
from PyQt5 import QtWidgets, uic
from PyQt5.QtMultimedia import *
from PyQt5.QtMultimediaWidgets import *
import os
import sys
import time

In [2]:

# https://github.com/citrusvanilla/horizon_detection
# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file 'process.ui'
#
# Created by: PyQt5 UI code generator 5.11.3
#
# WARNING! All changes made in this file will be lost!
#
# Subscribe to PyShine Youtube channel for more detail! 

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog
from PyQt5.QtGui import QImage
import cv2, imutils
import time
import numpy as np
from PIL import Image

import pyzed.sl as sl

import numpy as np
import cv2
import pyshine as ps
from scipy.signal import savgol_filter

import time
import math

try:
    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
except Exception as e:
    print('Warning...',e)


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        # MainWindow.resize(1024, 720)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setMinimumSize(1024, 720)
        self.centralwidget.setObjectName("centralwidget")
        self.gridLayout_2 = QtWidgets.QGridLayout(self.centralwidget)
        self.gridLayout_2.setObjectName("gridLayout_2")
        self.horizontalLayout = QtWidgets.QHBoxLayout()
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setText("")
        # self.label.setPixmap(QtGui.QPixmap("images/H.png"))
        self.label.setObjectName("label")
        self.horizontalLayout.addWidget(self.label)
        self.gridLayout = QtWidgets.QGridLayout()
        self.gridLayout.setObjectName("gridLayout")
        # self.verticalSlider = QtWidgets.QSlider(self.centralwidget)
        # self.verticalSlider.setOrientation(QtCore.Qt.Vertical)
        # self.verticalSlider.setObjectName("verticalSlider")
        # self.gridLayout.addWidget(self.verticalSlider, 0, 0, 1, 1)
        # self.verticalSlider_2 = QtWidgets.QSlider(self.centralwidget)
        # self.verticalSlider_2.setOrientation(QtCore.Qt.Vertical)
        # self.verticalSlider_2.setObjectName("verticalSlider_2")
        # self.gridLayout.addWidget(self.verticalSlider_2, 0, 1, 1, 1)
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setObjectName("label_2")
        self.gridLayout.addWidget(self.label_2, 1, 0, 1, 1)
        # self.label_3 = QtWidgets.QLabel(self.centralwidget)
        # self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        # self.label_3.setObjectName("label_3")
        # self.gridLayout.addWidget(self.label_3, 1, 1, 1, 1)
        self.horizontalLayout.addLayout(self.gridLayout)
        self.gridLayout_2.addLayout(self.horizontalLayout, 0, 0, 1, 2)
        self.horizontalLayout_2 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_2.setObjectName("horizontalLayout_2")
        # self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        # self.pushButton.setObjectName("pushButton")
        # self.horizontalLayout_2.addWidget(self.pushButton)
        self.pushButton_2 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.horizontalLayout_2.addWidget(self.pushButton_2)
        self.gridLayout_2.addLayout(self.horizontalLayout_2, 1, 0, 1, 1)
        spacerItem = QtWidgets.QSpacerItem(313, 20, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.gridLayout_2.addItem(spacerItem, 1, 1, 1, 1)
        MainWindow.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        # self.verticalSlider.valueChanged['int'].connect(self.brightness_value)
        # self.verticalSlider_2.valueChanged['int'].connect(self.blur_value)
        self.pushButton_2.clicked.connect(self.loadImage)
        # self.pushButton.clicked.connect(self.savePhoto)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        # Added code here
        self.filename = 'Snapshot '+str(time.strftime("%Y-%b-%d at %H.%M.%S %p"))+'.png' # Will hold the image address location
        self.tmp = None # Will hold the temporary image for display
        self.brightness_value_now = 0 # Updated brightness value
        self.blur_value_now = 0 # Updated blur value
        self.fps=0
        self.started = False

    def loadImage(self):
        """ This function will load the camera device, obtain the image
            and set it to label using the setPhoto function
        """
        if self.started:
            self.started=False
            self.pushButton_2.setText('Start')	
        else:
            self.started=True
            self.pushButton_2.setText('Stop')
        
        cam = True # True for webcam
        # Create a ZED camera object
        zed = sl.Camera()
        # if cam:
        vid = cv2.VideoCapture(1)
        

        # Set configuration parameters
        # input_type = sl.InputType()
        # if len(sys.argv) >= 2 :
        #     input_type.set_from_svo_file(sys.argv[1])
        # init = sl.InitParameters(input_t=input_type)
        init = sl.InitParameters()
        # init.camera_resolution = sl.RESOLUTION.HD1080
        init.camera_fps = 30

        init.camera_resolution = sl.RESOLUTION.HD1080
        init.depth_mode = sl.DEPTH_MODE.PERFORMANCE
        init.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        err = zed.open(init)
        if err != sl.ERROR_CODE.SUCCESS :
            print(repr(err))
            zed.close()
            exit(1)

        self.image = sl.Mat()
        runtime = sl.RuntimeParameters()
        runtime.sensing_mode = sl.SENSING_MODE.STANDARD

        # Prepare new image size to retrieve half-resolution images
        image_size = zed.get_camera_information().camera_resolution
        image_size.width = image_size.width /2
        image_size.height = image_size.height /2

        # Declare your sl.Mat matrices
        image_zed = sl.Mat(image_size.width, image_size.height, sl.MAT_TYPE.U8_C4)
        depth_image_zed = sl.Mat(image_size.width, image_size.height, sl.MAT_TYPE.U8_C4)
        point_cloud = sl.Mat()

        key = ' '

        # else:
        #     vid = cv2.VideoCapture('video.mp4')
        
        cnt=0
        frames_to_count=20
        st = 0
        fps=0

        sensors_data = sl.SensorsData()
        self.compasshead = 0
        heading_list = []
        # magnetometer_data = sl.SensorsData.get_magnetometer_data()
        
        # while(vid.isOpened()):
        while(zed.grab(runtime) == sl.ERROR_CODE.SUCCESS and key != 113):
            QtWidgets.QApplication.processEvents()

            # Retrieve the left image, depth image in the half-resolution
            zed.retrieve_image(image_zed, sl.VIEW.LEFT, sl.MEM.CPU, image_size)
            zed.retrieve_image(depth_image_zed, sl.VIEW.DEPTH, sl.MEM.CPU, image_size)
            # Retrieve the RGBA point cloud in half resolution
            zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA, sl.MEM.CPU, image_size)

            # To recover data from sl.Mat to use it with opencv, use the get_data() method
            # It returns a numpy array that can be used as a matrix with opencv
            image_ocv = image_zed.get_data()
            depth_image_ocv = depth_image_zed.get_data()

            # cv2.imshow("Image", image_ocv)
            # cv2.imshow("Depth", depth_image_ocv)
            self.image = image_ocv

            zed.get_sensors_data(sensors_data, sl.TIME_REFERENCE.IMAGE) # Retrieve only frame synchronized data

            # Extract magnetometer data
            magnetometer_data = sensors_data.get_magnetometer_data()

            # # Retrieve uncalibrated magnetic field
            # self.magnetic_field = magnetometer_data.get_magnetic_field_uncalibrated();  
            # print(self.magnetic_field)
            
            # Retrieve calibrated magnetic field
            magnetic_field = magnetometer_data.get_magnetic_field_calibrated();
            # print(magnetometer_data.magnetic_heading_state)
            # print(magnetometer_data.magnetic_heading_accuracy)
            heading = round(magnetometer_data.magnetic_heading)
            if(heading < 0):
                heading = 360 + heading
            # if(abs(self.compasshead - heading) > 5):
            # heading_list.append(heading)
            #     print(heading_list)
            self.compasshead = heading
            # if(len(heading_list) >= 10):
            #     result = savgol_filter(heading_list, 10, 5) # window size 13, polynomial order 5
            #     heading_list.pop(0)
            #     # heading_list.pop(4)
            #     self.compasshead = round(result[5])
            #     # heading_list.append(self.compasshead)
            
            

            
            gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY) 
            try:
                faces = faceCascade.detectMultiScale(
                gray,
                scaleFactor=1.15,  
                minNeighbors=7, 
                minSize=(80, 80), 
                flags=cv2.CASCADE_SCALE_IMAGE)
                
                for (x, y, w, h) in faces:
                    cv2.rectangle(self.image, (x, y), (x + w, y + h), (10, 228,220), 5) 
            except Exception as e:
                pass
            
            if cnt == frames_to_count:
                try: # To avoid divide by 0 we put it in try except
                    # print(frames_to_count/(time.time()-st),'FPS') 
                    self.fps = round(frames_to_count/(time.time()-st)) 
                    
                    
                    st = time.time()
                    cnt=0
                except:
                    pass
            
            cnt+=1
            
            self.update()
            key = cv2.waitKey(1) & 0xFF
            if self.started==False:
                zed.close()
                break
                print('Loop break')

    def setPhoto(self,image):
        """ This function will take image input and resize it 
            only for display purpose and convert it to QImage
            to set at the label.
        """
        
        

        img_overlay_rgba = np.array(Image.open("compass-scale.png"))
        compass_arrow = np.array(Image.open("compass_arrow.png"))

        compass_arrow = imutils.resize(compass_arrow, height=50)

        aspect = self.centralwidget.frameGeometry().width() / self.centralwidget.frameGeometry().height()
        if (aspect <= 16/9):
            
            image = imutils.resize(image, width=self.centralwidget.frameGeometry().width())
            img_overlay_rgba = imutils.resize(img_overlay_rgba, width=self.centralwidget.frameGeometry().width())
        else:
            
            image = imutils.resize(image, height=self.centralwidget.frameGeometry().height())
            img_overlay_rgba = imutils.resize(img_overlay_rgba, height=self.centralwidget.frameGeometry().height())


        x = round((image.shape[1]-img_overlay_rgba.shape[1])/2)
        y = round((image.shape[0]-img_overlay_rgba.shape[0])/2)
        if y<0:
            y = 0
        y = round(y + image.shape[0] - 300)
        # instead of using immediate rotate, use a function to iterate the value based on the delta.
        img_overlay_rgba = imutils.rotate(img_overlay_rgba, self.compasshead)

        # Perform blending
        alpha_mask = img_overlay_rgba[:, :, 3] / 255.0
        img_result = image[:, :, :3].copy()
        img_overlay = img_overlay_rgba[:, :, :3]

        self.overlay_image_alpha(img_result, img_overlay, x, y, alpha_mask)
        image = img_result


        x = round((image.shape[1]-compass_arrow.shape[1])/2)
        y = image.shape[0] - 80
        # # print(yoff,xoff)

        # # use numpy indexing to place the resized image in the center of background image
        # result = image.copy()
        # result[y:y+compass_arrow.shape[1], x:x+compass_arrow.shape[0]] = compass_arrow
        # image = result

        # Perform blending
        alpha_mask = compass_arrow[:, :, 3] / 255.0
        img_result = image[:, :, :3].copy()
        img_overlay = compass_arrow[:, :, :3]

        self.overlay_image_alpha(img_result, img_overlay, x, y, alpha_mask)
        image = img_result

        frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = QImage(frame, frame.shape[1],frame.shape[0],frame.strides[0],QImage.Format_RGB888)
        self.label.setPixmap(QtGui.QPixmap.fromImage(image))
        
    def overlay_image_alpha(self,img, img_overlay, x, y, alpha_mask):
        """Overlay `img_overlay` onto `img` at (x, y) and blend using `alpha_mask`.

        `alpha_mask` must have same HxW as `img_overlay` and values in range [0, 1].
        """
        # Image ranges
        y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
        x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

        # Overlay ranges
        y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
        x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

        # Exit if nothing to do
        if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
            return

        # Blend overlay within the determined ranges
        img_crop = img[y1:y2, x1:x2]
        img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
        alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]
        alpha_inv = 1.0 - alpha

        img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

    def brightness_value(self,value):
        """ This function will take value from the slider
            for the brightness from 0 to 99
        """
        self.brightness_value_now = value
        # print('Brightness: ',value)
        self.update()
        
        
    def blur_value(self,value):
        """ This function will take value from the slider 
            for the blur from 0 to 99 """
        self.blur_value_now = value
        print('Blur: ',value)
        self.update()


    def changeBrightness(self,img,value):
        """ This function will take an image (img) and the brightness
            value. It will perform the brightness change using OpenCv
            and after split, will merge the img and return it.
        """
        hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
        lim = 255 - value
        v[v>lim] = 255
        v[v<=lim] += value
        final_hsv = cv2.merge((h,s,v))
        img = cv2.cvtColor(final_hsv,cv2.COLOR_HSV2BGR)
        return img
        
    def changeBlur(self,img,value):
        """ This function will take the img image and blur values as inputs.
            After perform blur operation using opencv function, it returns 
            the image img.
        """
        kernel_size = (value+1,value+1) # +1 is to avoid 0
        img = cv2.blur(img,kernel_size)
        return img

    def update(self):
        """ This function will update the photo according to the 
            current values of blur and brightness and set it to photo label.
        """
        img = self.changeBrightness(self.image,self.brightness_value_now)
        # img = self.changeBlur(img,self.blur_value_now)


        # Here we add display text to the image
        # text  =  'FPS: '+str(self.fps)
        # img = ps.putBText(img,text,text_offset_x=20,text_offset_y=30,vspace=20,hspace=10, font_scale=1.0,background_RGB=(10,20,222),text_RGB=(255,255,255))
        text = str(self.compasshead)
        img = ps.putBText(img,text,text_offset_x=int(self.image.shape[1]/2),text_offset_y=30,vspace=20,hspace=10, font_scale=1.0,background_RGB=(228,20,222),text_RGB=(255,255,255))
        # text  =  f"Brightness: {self.brightness_value_now}"
        # img = ps.putBText(img,text,text_offset_x=80,text_offset_y=425,vspace=20,hspace=10, font_scale=1.0,background_RGB=(20,210,4),text_RGB=(255,255,255))
        # text  =  f'Blur: {self.blur_value_now}: '
        # img = ps.putBText(img,text,text_offset_x=self.image.shape[1]-200,text_offset_y=425,vspace=20,hspace=10, font_scale=1.0,background_RGB=(210,20,4),text_RGB=(255,255,255))


        self.setPhoto(img)

    def savePhoto(self):
        """ This function will save the image"""
        self.filename = 'Snapshot '+str(time.strftime("%Y-%b-%d at %H.%M.%S %p"))+'.png'
        cv2.imwrite(self.filename,self.tmp)
        print('Image saved as:',self.filename)


    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Zed Navigation"))
        self.pushButton_2.setText(_translate("MainWindow", "Start"))
        # self.label_2.setText(_translate("MainWindow", "Brightness"))
        # self.label_3.setText(_translate("MainWindow", "Blur"))
        # self.pushButton.setText(_translate("MainWindow", "Take picture"))


if __name__ == "__main__":
	import sys
	app = QtWidgets.QApplication(sys.argv)
	MainWindow = QtWidgets.QMainWindow()
	ui = Ui_MainWindow()
	ui.setupUi(MainWindow)
	MainWindow.show()
	sys.exit(app.exec_())



Warning! No module named 'sounddevice'
Warning! No module named 'matplotlib'
Warning! No module named 'keras'
